In [1]:
import sys; sys.path.append('..'); from cadence.imports import *; import cadence as cd

In [5]:
import sys; sys.path.append('../../prosodic')
import prosodic as p
p.config['print_to_screen']=False

In [11]:
txt="""In Xanadu did Kubla Khan
A stately pleasure-dome decree:
Where Alph, the sacred river, ran
Through caverns measureless to man
   Down to a sunless sea.
"""
line=txt.split('\n')[0]

In [14]:
p.config

{'add_elided_pronunciations': 1,
 'en_TTS_ENGINE': 'espeak',
 'en_TTS_cache': 1,
 'folder_tagged_samples': 'tagged_samples/',
 'lang': 'en',
 'line_headedness': 'ws',
 'line_maxsylls': 60,
 'linebreak': 'line',
 'linelen': 60,
 'maxent_gradient_norm_tolerance': 1e-06,
 'maxent_max_epochs': 10000,
 'maxent_negative_weights_allowed': False,
 'maxent_step_size': 0.1,
 'meter': <Meter.Meter at 0x1692abe80>,
 'num_bounded_parses_to_store': 100,
 'output': 'ipa',
 'output_en': 'ipa',
 'output_fi': 'orth',
 'parse_maxsec': 30,
 'parse_using_metrical_tree': 0,
 'path_corpora': '/Users/ryan/prosodic_data/corpora',
 'path_meters': '/Users/ryan/prosodic_data/meters',
 'path_nlp_data': '/Users/ryan/prosodic_data/nlp_libraries',
 'path_results': '/Users/ryan/prosodic_data/results',
 'path_tagged_samples': '/Users/ryan/prosodic_data/tagged_samples',
 'print_to_screen': False,
 'resolve_optionality': 1,
 'tokenizer': '[^\\s+]+',
 'path_prosodic_data': '/Users/ryan/prosodic_data',
 'meters': {'iambic_

In [42]:
l=parse_prosodic('Bird bird bird bird', constraints=['*w/peak','*w/stressed','*s/unstressed','*f-res'])

In [43]:
prs=l.bestParse()
pos=prs.positions[0]
prs.constraintScores

{[*strength.w=>-p]: 0.0,
 [*stress.w=>-p]: 2.0,
 [*stress.s=>-u]: 0.0,
 [*footmin-f-resolution]: 0}

In [33]:
for posx in prs.positions: print(posx.constraintScores)

{[*strength.w=>-p]: 0, [*stress.w=>-p]: 0, [*stress.s=>-u]: 0.0}
{[*strength.w=>-p]: 0.0, [*stress.w=>-p]: 0.0, [*stress.s=>-u]: 0}
{[*strength.w=>-p]: 0, [*stress.w=>-p]: 0, [*stress.s=>-u]: 0.0}
{[*strength.w=>-p]: 0.0, [*stress.w=>-p]: 0.0, [*stress.s=>-u]: 0}
{[*strength.w=>-p]: 0, [*stress.w=>-p]: 0, [*stress.s=>-u]: 0.0}
{[*strength.w=>-p]: 0.0, [*stress.w=>-p]: 0.0, [*stress.s=>-u]: 0}


In [10]:
p.config['meters']['default_english'].constraints

[[*strength.w=>-p],
 [*stress.s=>-u],
 [*stress.w=>-p],
 [*footmin-w-resolution],
 [*footmin-f-resolution]]

In [9]:
# p.Meter

Metrically parsing lines (+word IPA combinations) [x8]: 100%|██████████| 4/4 [00:04<00:00,  1.01s/it]


Index(['stanza_i', 'line_i', 'line_str', 'parse_rank', 'combo_i',
       'combo_stress', 'combo_ipa', 'parse_i', 'parse', 'parse_str',
       'parse_pos_i', 'parse_pos', 'word_i', 'word_str', 'word_ipa_i',
       'word_ipa', 'syll_i', 'combo_syll_i', 'syll_str', 'syll_ipa',
       'syll_stress', 'syll_weight', 'parse_syll_i', 'parse_syll', '*total',
       '*w-res', '*f-res', '*w/stressed', '*s/unstressed', '*clash', '*w/peak',
       '*lapse', 'combo_num_syll', 'is_funcword', 'is_heavy', 'is_light',
       'is_peak', 'is_s', 'is_stressed', 'is_syll', 'is_trough',
       'is_unstressed', 'is_w', 'parse_num_pos', 'parse_num_syll',
       'prom_strength', 'prom_stress', 'prom_weight'],
      dtype='object')

In [46]:
def parse_prosodic(txt_or_txtdf, lang='en',
                   meter='default_english', use_espeak=True,
                   constraints=None,constraint_weights=None):
    import prosodic as p
    p.config['resolve_optionality']=0
    p.config['print_to_screen']=True
    p.config['en_TTS_ENGINE']='espeak' if use_espeak else 'none'
    
    meter_obj=p.config['meters'][meter]
    config=default_config=meter_obj.config
    if constraints is not None:
        constraints_pros=[
            f'{constraint_names_in_prosodic[c]}/{constraint_weights[i] if constraint_weights is not None else 1}'
            for c in constraints
        ]
        config['constraints']=constraints_pros
        meter_obj=p.Meter(config)
    
    
    txtdf = scan(txt_or_txtdf) if type(txt_or_txtdf)==str else txt_or_txtdf
    txtdf = txtdf.sort_index()
    
    o=[]
    for combodf in iter_combos(txtdf):
        words=[]
        for wi,wdf in combodf.reset_index().groupby('word_i'):
            ipa_str='.'.join(wdf.syll_ipa)
            sylls_text=list(wdf.syll_str)
            token = wdf.iloc[0].word_str
            word_obj = p.dict[lang].make((ipa_str,sylls_text), token)
            wordtoken_obj = p.WordToken([word_obj], token)
            words.append(wordtoken_obj)
        
        l=p.Line()
        l.children = words
        
        l.parse(meter=meter_obj)
        return get_data_from_line(l,meter_obj)
        
        

In [47]:
# def get_data_from_line1(line,meter):
#     bp=line.bestParse(meter)
#     ap=line.allParses(meter)
#     output_dict={}
#     output_dict['prosodic_line']=line.txt #.encode('utf-8',errors='ignore')
#     output_dict['parse_str']=bp.posString(viols=True) if bp else ''
#     meter_str=output_dict['meter']=bp.str_meter() if bp else ''
#     output_dict['num_parses']=len(ap)
#     output_dict['*total'] = bp.totalCount if bp else ''
#     output_dict['score_viols'] = bp.score() if bp else ''
#     output_dict['num_sylls']=bp.num_sylls if bp else ''
#     output_dict['num_words']=len(line.words())
#     for c in meter.constraints:
#         sumviol = sum([parse.constraintCounts[c] if c in parse.constraintCounts else 0 for parse in ap])
#         output_dict[c.name_weight+'_bestparse']=bp.constraintCounts[c] if bp and c in bp.constraintCounts else 0
#         output_dict[c.name_weight+'_allparse_sum']=sumviol if sumviol else 0
    
    
    
    
#     ## store phonological constraint stats
# #     output_dict['prosodic_stress']=stress_str
# #     output_dict['prosodic_weight']=weight_str
# #     output_dict['prosodic_sonority']=sonority_str
#     output_dict['num_monosylls']=len([w for w in line.words() if w.numSyll==1])
#     return output_dict

In [ ]:
def get_data_from_line(line,viols_in_parse_str=True):
    o=[]
    for parse in line.allParses():
        parse_str=parse.posString(viols=viols_in_parse_str)
        for pos_i,pos in enumerate(parse.positions):
            for slot_i,slot in enumerate(parse.slots):
                o+=[]
            

In [41]:
for combodf in iter_combos(scan(line)): pass
combo_cols=set(combodf.reset_index().columns)
combo_cols
combodf_parsed=parse_combo(combodf).reset_index()
combo_cols_parsed = set(combodf_parsed.columns)
cols_weneed=combo_cols_parsed - combo_cols
combodf_parsed[cols_weneed]

,*clash,*w/peak,parse_pos,is_w,*w/stressed,parse_syll,parse_i,*total,*f-res,*w-res,*s/unstressed,parse_num_pos,parse_str,parse_syll_i,*lapse,is_s,parse_num_syll,parse,parse_pos_i
0,0,0,w,1,1,w,0,1.0,NaN,NaN,0,8,in|XA|na|DU|did|KU|bla|KHAN,0,0,0,8,wswswsws,0
1,0,0,s,0,0,s,0,1.0,NaN,NaN,1,8,in|XA|na|DU|did|KU|bla|KHAN,1,0,1,8,wswswsws,1
2,0,1,w,1,1,w,0,2.0,NaN,NaN,0,8,in|XA|na|DU|did|KU|bla|KHAN,2,0,0,8,wswswsws,2
3,0,0,s,0,0,s,0,1.0,NaN,NaN,1,8,in|XA|na|DU|did|KU|bla|KHAN,3,0,1,8,wswswsws,3
4,0,0,w,1,0,w,0,0.0,NaN,NaN,0,8,in|XA|na|DU|did|KU|bla|KHAN,4,0,0,8,wswswsws,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,0,0,ww,1,0,w,67,0.0,NaN,NaN,0,4,IN.XA|na.du|DID.KU|bla.khan,3,0,0,8,sswwssww,1
540,0,0,ss,0,0,s,67,4.0,1.0,1.0,1,4,IN.XA|na.du|DID.KU|bla.khan,4,1,1,8,sswwssww,2
541,0,0,ss,0,0,s,67,2.0,1.0,1.0,0,4,IN.XA|na.du|DID.KU|bla.khan,5,0,1,8,sswwssww,2
542,0,0,ww,1,0,w,67,0.0,NaN,NaN,0,4,IN.XA|na.du|DID.KU|bla.khan,6,0,0,8,sswwssww,3


In [41]:
cols_weneed=set(parse_cols) - set(combo_cols)
cols_weneed

{'*clash',
 '*f-res',
 '*lapse',
 '*s/unstressed',
 '*total',
 '*w-res',
 '*w/peak',
 '*w/stressed',
 'is_s',
 'is_w',
 'parse',
 'parse_i',
 'parse_num_pos',
 'parse_num_syll',
 'parse_pos',
 'parse_pos_i',
 'parse_rank',
 'parse_str',
 'parse_syll',
 'parse_syll_i'}

In [61]:
def parse_phons():
    output_dict['[*clash_across]']=counts(stress_str,'P#P') + counts(stress_str,'P#S') + counts(stress_str,'S#P') + counts(stress_str,'S#S')
    output_dict['[*clash_within]']=counts(stress_str,'PP') + counts(stress_str,'PS') + counts(stress_str,'SP') + counts(stress_str,'SS')
    output_dict['[*clash_across_primary]']=counts(stress_str,'P#P')
    output_dict['[*clash_within_primary]']=counts(stress_str,'PP')
    output_dict['[*lapse_across]']=counts(stress_str,'U#U')
    output_dict['[*lapse_within]']=counts(stress_str,'UU')
    output_dict['[*WSP]']=0
    output_dict['[*PEAKPROM]']=0
    output_dict['[*High_Stress]']=0
    output_dict['[*Low_Unstress]']=0
    output_dict['[*High_Strong]']=0
    output_dict['[*Low_Weak]']=0
    for s,w,hml,mtr in zip(stress_str,weight_str,sonority_str,meter_str):
        if s=='U' and w=='H':
            output_dict['[*WSP]']+=1
        if (s=='P' or s=='S') and w=='L':
            output_dict['[*PEAKPROM]']+=1
        
        if hml=='H' and s in {'P','S'}:
            output_dict['[*High_Stress]']+=1
        if hml=='L' and s=="U":
            output_dict['[*Low_Unstress]']+=1
        
        if hml=='H' and mtr == 's':
            output_dict['[*High_Strong]']+=1
        if hml=='L' and mtr == 'w':
            output_dict['[*Low_Weak]']+=1